**Avant de débuter ce TP** :

1. **Changez le type d'exécution sur Google Colab** : `Exécution > Modifiez le type d'exécution > T4 GPU`
2. **Installez les paquets ci-dessous** :

In [ ]:
! pip install diffusers

# Inférence avec des modèles pré-entraînés

### Introduction

De nombreux modèles pré-entraînés sont publiquement disponibles sur internet.
Certains sont assez généraux, mais la plupart sont des des modèles affinés sur des tâches spécifiques.

Une interface de programmation d'application (API pour *application programming interface*) est [Hugging Face](https://huggingface.co).
Plusieurs paquets Python, tels que [transformers](https://huggingface.co/docs/transformers/index) et [diffusers](https://huggingface.co/docs/diffusers/index), ont été développés dans cet écosystème et mettent à disposition des modèles pré-entraînés.

La liste de tous les modèles sont disponibles sur Hugging Face est disponible [ici](https://huggingface.co/models).
À ce jour, il y a plus d'un million de modèles pré-entraînés disponibles.
Il est facile de filtrer la liste des modèles en fonction de certains critères (types de tâche, bibliothèques, jeux de données, langues, licences, etc.)

Dans ce notebook, on ne va pas entraîner de modèle.
On va seulement effectuer de l'inférence avec des modèles pré-entraînés et publiquement disponibles.
L'objectif de ce notebook est d'apprendre à chercher des modèles pré-entraînés et de les utiliser sans utiliser une interface graphique (comme ChatGPT).
L'autre avantage, avec des modèles pré-entraînés et publiquement disponibles, est que la seule limite d'utilisation est la puissance de calcul.

### Identification

Un certain nombre de modèles pré-entraînés, bien qu'accessibles publiquement, nécessite de s'authentifier sur Hugging Face.
Il est donc nécessaire de [créer un compte](https://huggingface.co/join), de confirmer son adresse mail, puis de [se connecter](https://huggingface.co/login) si nécessaire, d'aller dans [`Setting/Access Tokens`](https://huggingface.co/settings/tokens) (**choisissez `Read` pour le type de token**), de créer un nouveau token et de le copier.
Enfin, pour se connecter dans un notebook, il suffit de coller le token dans la barre qui s'affiche en exécutant la cellule suivante :

In [ ]:
from huggingface_hub import login

login()

## Systèmes de questions-réponses

Un [système de questions-réponses](https://fr.wikipedia.org/wiki/Systèmes_de_questions-réponses) est un système informatique permettant de répondre automatiquement à des questions posées par des humains, lors d'un échange fait en langue naturelle.

On va utiliser le modèle [RoBERTa-base-squad2](https://huggingface.co/deepset/roberta-base-squad2), qui est le modèle [RoBERTa](https://huggingface.co/roberta-base) mais affiné sur le jeu de données [SQuAD2.0](https://huggingface.co/datasets/squad_v2).
Ce modèle a été entraîné sur des paires de questions et de réponses, avec notamment des questions pour lesquelles il est impossible de répondre.

L'entrée du modèle est un dictionnaire avec les paires suivantes de clés et de valeurs :
* la clé `'question'` dont la valeur est une chaîne de caractères correspondant à la question,
* la clé `'context'` dont la valeur est une chaîne de caractères contenant l'information nécessaire pour répondre à la question.

La sortie du modèle est un dictionnaire avec les paires suivantes de clés et de valeurs :
* la clé `'score'` dont la valeur est score de confiance du modèle dans sa réponse,
* la clé `'start'` dont la valeur est l'indice du premier caractère de la réponse,
* la clé `'start'` dont la valeur est l'indice du dernier caractère de la réponse,
* la clé `'answer'` dont la valeur est la réponse renvoyée par le modèle.

Voici un exemple d'utilisation de ce modèle :

```python
>>> from transformers import pipeline
>>> question_answering = pipeline(
...     'question-answering',
...     model='deepset/roberta-base-squad2',
...     tokenizer="deepset/roberta-base-squad2"
... )
>>> question_answering({
...     'question': "How many planets are there in the solar system?",
...     'context': (
...         "The eight planets of the solar system are Mercury, Venus, Earth, "
...         "Mars, Jupiter, Saturn, Uranus, and Neptune."
...     )
... })
{'score': 0.7999436855316162, 'start': 4, 'end': 9, 'answer': 'eight'}
```

Une limite importante du modèle est que la réponse doit se trouver explicitement (c'est-à-dire sous la forme de mots) dans le contexte.
Par exemple, si on enlève le mot `'eight'` du contexte, la réponse est décevante (mais le score de confiance est également beaucoup plus faible).

```python
>>> question_answering({
...     'question': "How many planets are there in the solar system?",
...     'context': (
...         "The planets of the solar system are Mercury, Venus, Earth, "
...         "Mars, Jupiter, Saturn, Uranus, and Neptune."
...     )
... })
{'score': 0.01189940981566906,
 'start': 46,
 'end': 102,
 'answer': 'Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune'}
```

### Exercice

Utilisez ce modèle avec vos propres paires de questions et de contextes.

In [ ]:
from transformers import pipeline

question_answering = pipeline(
    'question-answering',
    model='deepset/roberta-base-squad2',
    tokenizer="deepset/roberta-base-squad2"
)

In [ ]:
question_answering({
    'question': "How many planets are there in the solar system?",
    'context': (
        "The eight planets of the solar system are Mercury, Venus, Earth, "
        "Mars, Jupiter, Saturn, Uranus, and Neptune."
    )
})

In [ ]:
question_answering({
    'question': "How many planets are there in the solar system?",
    'context': (
        "The planets of the solar system are Mercury, Venus, Earth, "
        "Mars, Jupiter, Saturn, Uranus, and Neptune."
    )
})

In [ ]:
question_answering({
    'question': "Where am I teaching?",
    'context': "My name is Johann and I'm a an assistant professor of computer science at ENSAI."
})

In [ ]:
question_answering({
    'question': "What is Geralt killing?",
    'context': (
        "Geralt is a Witcher, a man whose magic powers, enhanced by long "
        "training and a mysterious elixir, have made him a brilliant fighter "
        "and a merciless hunter. Yet he is no ordinary killer: his targets are "
        "the multifarious monsters and vile fiends that ravage the land "
        "and attack the innocent."
    )
})

In [ ]:
question_answering({
    'question': "Where is Aloy going?",
    'context': (
        "Horizon Forbidden West continues the story of Aloy, a young hunter of the "
        "Nora tribe and a clone of the Old World scientist Elisabet Sobeck, as she "
        "leads a band of companions on a quest to the frontier region known as the "
        "Forbidden West to find the source of a mysterious plague that kills all it "
        "infects. On her journey across these lands, Aloy encounters new regions "
        "ravaged by massive storms and deadly machines, and conflicts with a tribe "
        "of nomadic raiders, some of which have tamed the machines as war mounts. "
        "She discovers a vast array of environments and ecosystems, including lush "
        "valleys, dry deserts, snowy mountains, tropical beaches, ruined cities, "
        "and underwater settings."
    )
})

## Génération d'images à partir de texte

Le [*prompt art*](https://fr.wikipedia.org/wiki/Prompt_art) ([*text-to-image model*](https://en.wikipedia.org/wiki/Text-to-image_model) est une technologie qui consiste à créer des oeuvres d'art au moyen de l'intelligence artificielle en fournissant des mots clés en entrée du modèle.
De tels modèles ont commencé à être développés au milieu des années 2010.
Aujourd'hui, les images renvoyées par des modèles tels que [DALL-E 2](https://openai.com/dall-e-2), [Imagen](https://imagen.research.google), [Stable Diffusion](https://stability.ai/stable-diffusion), and [Midjourney](https://www.midjourney.com/showcase/recent/) sont proches de vraies photographies ou d'images générées manuellement par un•e artiste humain•e.

### Images de mosaïques

Nous allons utiliser le modèle [MosaicArt](https://huggingface.co/Guizmus/MosaicArt) pour générer des images de mosaïque.
Ce modèle est basé sur le modèle [Stable Diffusion 2](https://huggingface.co/stabilityai/stable-diffusion-2-1), mais affiné en utilisant [DreamBooth](https://dreambooth.github.io).

Ci-dessous se trouve une illustration d'images générées par ce modèle.

[<img src="https://huggingface.co/Guizmus/MosaicArt/resolve/main/showcase.jpg" width="450"/>](https://huggingface.co/Guizmus/MosaicArt/resolve/main/showcase.jpg)

Voici un exemple d'utilisation de ce modèle. Le token `"Mosaic Art"` au début du *prompt* indique que l'on veut générer des images de mosaïque.

```python
from diffusers import StableDiffusionPipeline
import torch


# Define the generator
mosaic_art_generator = StableDiffusionPipeline.from_pretrained(
    "Guizmus/MosaicArt", torch_dtype=torch.float16
).to("cuda")

# Generate an image given a prompt and show it
prompt = "Mosaic Art dog on the moon"
image = mosaic_art_generator(prompt).images[0]
image
```

### Exercice

Utilisez ce modèle pré-entraîné pour générer des images représentant des mosaïques.

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

mosaic_art_generator = StableDiffusionPipeline.from_pretrained(
    "Guizmus/MosaicArt", torch_dtype=torch.float16
).to("cuda")

In [ ]:
prompt = "Mosaic Art dog on the moon"
image = mosaic_art_generator(prompt).images[0]
image

In [ ]:
prompt = "Mosaic Art cat in a forest"
image = mosaic_art_generator(prompt).images[0]
image

In [ ]:
prompt = "Mosaic Art lions in the wild"
image = mosaic_art_generator(prompt).images[0]
image

### Images avec le style d'Hergé

[Hergé](https://fr.wikipedia.org/wiki/Hergé) est un auteur de bande dessinée belge, principalement connu pour *Les Aventures de Tintin*.
Vous allez générer des images avec le style d'Hergé.

Là encore, le principe est le même.
Il s'agit du modèle [Stable Diffusion](https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0), mais affiné en utilisant [DreamBooth](https://dreambooth.github.io) pour générer des images avec le style d'Hergé.
Vous trouverez les 10 images originales utilisées pour l'affinage [ici](https://huggingface.co/sd-dreambooth-library/herge-style).

Voici un exemple d'utilisation de ce modèle. Le mot clé dans le prompt est le token `herge_style`. 

```python
from diffusers import AutoPipelineForText2Image
import torch

# Load the model
pipeline = AutoPipelineForText2Image.from_pretrained(
    "sd-dreambooth-library/herge-style", torch_dtype=torch.float16
).to("cuda")

# Define the prompt and generate an image
prompt = (
    "A cute herge_style brown bear eating a slice of pizza, "
    "stunning color scheme, masterpiece, illustration"
)
image = pipeline(prompt).images[0]
image
```

### Exercice

Utilisez ce modèle pré-entraîné pour générer des images avec le style d'Hergé.

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch


pipeline = AutoPipelineForText2Image.from_pretrained(
    "sd-dreambooth-library/herge-style", torch_dtype=torch.float16
).to("cuda")

In [ ]:
prompt = (
    "A cute herge_style brown bear eating a slice of pizza, "
    "stunning color scheme, masterpiece, illustration"
)
image = pipeline(prompt).images[0]
image

In [ ]:
prompt = (
    "A white herge_style dog biting the leg of a man, "
    "stunning color scheme, masterpiece, illustration"
)
image = pipeline(prompt).images[0]
image

In [ ]:
prompt = (
    "A black herge_style cat walking in a garden, "
    "stunning color scheme, masterpiece, illustration"
)
image = pipeline(prompt).images[0]
image

### Arts graphiques

Les [arts graphiques](https://fr.wikipedia.org/wiki/Arts_graphiques) correspondent à l'une des catégories traditionnelles de disciplines des beaux-arts.
Vous allez générer des images d'arts graphiques.

Ce modèle est basé sur le modèle [Stable Diffusion 2](https://huggingface.co/stabilityai/stable-diffusion-2-1).
Ci-dessous se trouve une illustration d'images générées par ce modèle.

[<img src="https://huggingface.co/n6ai/graphic-art/resolve/main/.huggingface/preview.png" width="650"/>](https://huggingface.co/n6ai/graphic-art/resolve/main/.huggingface/preview.png)

Voici un exemple d'utilisation de ce modèle.
```python
import torch
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained(
    "n6ai/graphic-art", torch_dtype=torch.float16
).to("cuda")

prompt = "a young man with black hair wearing a mask"
image = pipe(prompt).images[0]
image
```

### Exercice

Utilisez ce modèle pré-entraîné pour générer des images d'arts graphiques.

In [ ]:
import torch
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained(
    "n6ai/graphic-art", torch_dtype=torch.float16
).to("cuda")

In [ ]:
prompt = "a young man with black hair wearing a mask"
image = pipe(prompt).images[0]
image

In [ ]:
prompt = "an anime ninja"
image = pipe(prompt).images[0]
image

In [ ]:
prompt = "view of Paris with the Eiffel tower in the background"
image = pipe(prompt).images[0]
image

## Traduction

La [traduction](https://fr.wikipedia.org/wiki/Traduction) est le fait de faire passer un texte rédigé dans une langue dans une autre langue.
La [traduction automatique](https://fr.wikipedia.org/wiki/Traduction_automatique) désigne la traduction brute d'un texte entièrement réalisée par un ou plusieurs programmes informatiques.
La [traduction automatique neuronale](https://fr.wikipedia.org/wiki/Traduction_automatique_neuronale) est une approche de traduction automatique qui utilise un réseau neuronal artificiel pour prédire la probabilité d'une séquence de mots, généralement en modélisant puis en traduisant des phrases entières avec un seul modèle.

Les traducteurs automatiques tels que [DeepL Translator](https://www.deepl.com/en/translator) et [Goole Translate](https://translate.google.com) sont basés sur des réseaux de neur based on artifical neural networks.

Nous allons utiliser une plus petite version du modèle pré-entraîné [Text-To-Text Transfer Transformer](https://huggingface.co/t5-small) (T5-small).
Bien que ce modèle puisse être utilisé pour de [nombreuses tâches du traitement automatique des langues](https://huggingface.co/docs/transformers/model_doc/t5#resources), nous allons nous focaliser sur la traduction uniquement.
Ce plus petit modèle fonctionne pour quatre langues : anglais, français, roumain et allemand.

Voici un exemple d'utilisation de ce modèle :

```python
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the model
tokenizer_t5 = T5Tokenizer.from_pretrained("t5-small")
model_t5 = T5ForConditionalGeneration.from_pretrained("t5-small")

# Define the input
sentence = "Iris is a flowering plant genus of 310 accepted species with showy flowers."

# Define the task
input_ids = tokenizer(f"translate English to French: {sentence}", return_tensors="pt").input_ids

# Generate the output
outputs = model.generate(input_ids, max_new_tokens=50)

# Print the output
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
```

Porter une attention particulière à deux éléments :

* Le début du *prompt* indique la tâche : `'translate English to French: '` pour traduire de l'anglais vers le français.
* Il semblerait que seul l'anglais soit disponible pour la langue de la phrase à traduire. Donc il y a trois tâches possible : anglais > français, anglais > allemand et anglais > roumain.
* `max_new_tokens` indique le nombre maximum de tokens à générer.

### Exercice

Utilisez ce modèle pré-entraîné pour traduire des phrases d'une langue à une autre parmi les langues disponibles avec ce modèle.

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer_t5 = T5Tokenizer.from_pretrained("t5-small")
model_t5 = T5ForConditionalGeneration.from_pretrained("t5-small")

In [ ]:
sentence = "Iris is a flowering plant genus of 310 accepted species with showy flowers."

input_ids = tokenizer_t5(f"translate English to French: {sentence}", return_tensors="pt").input_ids
outputs = model_t5.generate(input_ids, max_new_tokens=50)
print(tokenizer_t5.decode(outputs[0], skip_special_tokens=True))

In [ ]:
sentence = ("Time flies by so fast.")

input_ids = tokenizer_t5(f"translate English to French: {sentence}", return_tensors="pt").input_ids
outputs = model_t5.generate(input_ids, max_new_tokens=50)
print(tokenizer_t5.decode(outputs[0], skip_special_tokens=True))

In [ ]:
sentence = ("Time flies by so fast.")

input_ids = tokenizer_t5(f"translate English to German: {sentence}", return_tensors="pt").input_ids
outputs = model_t5.generate(input_ids, max_new_tokens=50)
print(tokenizer_t5.decode(outputs[0], skip_special_tokens=True))

In [ ]:
sentence = ("Time flies by so fast.")

input_ids = tokenizer_t5(f"translate English to Romanian: {sentence}", return_tensors="pt").input_ids
outputs = model_t5.generate(input_ids, max_new_tokens=50)
print(tokenizer_t5.decode(outputs[0], skip_special_tokens=True))

## Génération automatique de musique

L'intelligence artificielle est de plus en plus utilisée dans l'industrie musicale.

Nous allons utiliser ici le modèle [MAGNeT](https://huggingface.co/facebook/magnet-small-10secs), et plus précisément sa version la plus petite, qui permet de générer des musiques de 10 secondes.
L'utilisation de ce modèle nécessite l'installation de la bibiothèque [`audiocraft`](https://github.com/facebookresearch/audiocraft).

Voici un exemple d'utilisation de ce modèle :
```python
from audiocraft.models import MAGNeT
from audiocraft.data.audio import audio_write

# Load the model
model = MAGNeT.get_pretrained("facebook/magnet-small-10secs")

# Define the styles of music that we want to generate
descriptions = ["happy rock", "energetic EDM", "heavy metal", "classical music"]

# Generate the samples
wav = model.generate(descriptions)

# Save the samples as WAV files
for idx, (description, one_wav) in enumerate(zip(descriptions, wav)):
    audio_write(f'{description} {idx}', one_wav.cpu(), model.sample_rate, strategy="loudness")
```

Vous pouvez également directment lire des fichiers audio dans un notebook :
```python
import IPython

IPython.display.display(IPython.display.Audio("happy rock 0.wav"))
IPython.display.display(IPython.display.Audio("energetic EDM 1.wav"))
IPython.display.display(IPython.display.Audio("heavy metal 2.wav"))
IPython.display.display(IPython.display.Audio("classical music 3.wav"))
```

### Exercice

Utilisez ce modèle pré-entraîné pour générer des échantillons de 10 secondes de différents styles de musique.

In [ ]:
! pip install git+https://github.com/facebookresearch/audiocraft.git

In [ ]:
from audiocraft.models import MAGNeT
from audiocraft.data.audio import audio_write

model = MAGNeT.get_pretrained("facebook/magnet-small-10secs")

In [ ]:
descriptions = ["happy rock", "energetic EDM", "heavy metal", "classical music"]

wav = model.generate(descriptions)

for idx, (description, one_wav) in enumerate(zip(descriptions, wav)):
    audio_write(f'{description} {idx}', one_wav.cpu(), model.sample_rate, strategy="loudness")

In [ ]:
import IPython

IPython.display.display(IPython.display.Audio("happy rock 0.wav"))
IPython.display.display(IPython.display.Audio("energetic EDM 1.wav"))
IPython.display.display(IPython.display.Audio("heavy metal 2.wav"))
IPython.display.display(IPython.display.Audio("classical music 3.wav"))